Reference: https://github.com/adeshpande3/Tensorflow-Programs-and-Tutorials/blob/master/CNNs%20with%20Noisy%20Labels.ipynb.

This code is to test out whether a CNN that is trained on noisy labels is still able to achieve good accuracy.
______
However, my result is not good.

In [1]:
import tensorflow as tf
import numpy as np
import random
import datetime
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings('ignore')

/Users/quansun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load data
mnist = input_data.read_data_sets('./Basic/mnist', one_hot=True)

tf.reset_default_graph()

x = tf.placeholder(tf.float32,shape=[None,784])
y_ = tf.placeholder(tf.float32,shape=[None,10])
x_image = tf.reshape(x,[-1,28,28,1])

# first conv layer
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1,shape=[32]))
h_conv1 = tf.nn.conv2d(x_image,W_conv1,strides=[1,1,1,1],padding='SAME') + b_conv1
h_conv1 = tf.nn.relu(h_conv1)
h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# second conv layer
W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64],stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1,shape=[64]))
h_conv2 = tf.nn.conv2d(h_pool1,W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2
h_conv2 = tf.nn.relu(h_conv2)
h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# first fully commected layer
W_fc1 = tf.Variable(tf.truncated_normal([7*7*64,1024],stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[1024]))
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(h_fc1)

# dropout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# second fully connected layer
W_fc2 = tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y,labels=y_))
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize(loss)

# Session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./Basic/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./Basic/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./Basic/mnist/t10k-images-idx3-ubyte.gz
Extracting ./Basic/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# nosiy labels settings
changeLabelProbability = 0.5

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = 'tensorboard/' + 'ChangeLabelProbability:' + str(changeLabelProbability) + '/'
writer = tf.summary.FileWriter(logdir, sess.graph)

In [4]:
batch_size = 64
epochs = 10000

for epoch in range(epochs):
    x_batch, y_batch = mnist.train.next_batch(batch_size)
    
    if epoch % 10 == 0:
        summary = sess.run(merged, feed_dict={x:x_batch, y_:y_batch, keep_prob:1.0})
        writer.add_summary(summary, epoch)
        
    if epoch % 1000 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={x:x_batch,y_:y_batch,keep_prob:1.0})
        
        print('step %d, training accuracy %g' % (epoch, train_accuracy))
        
        # noisy labels
        if (random.random() < changeLabelProbability):
            for batch in range(batch_size):
                originalTrainingLabel = np.argmax(y_batch[batch])
                validOtherChoices = list(range(0,originalTrainingLabel)) + list(range(originalTrainingLabel+1,10))
                newTrainingLabel = random.choice(validOtherChoices)
                y_batch[batch] = np.zeros(10)
                y_batch[batch][newTrainingLabel] = 1
                
        train_step.run(session=sess,feed_dict={x:x_batch,y_:y_batch,keep_prob:0.2})

step 0, training accuracy 0.09375
step 1000, training accuracy 0.140625
step 2000, training accuracy 0.109375
step 3000, training accuracy 0.234375
step 4000, training accuracy 0.203125
step 5000, training accuracy 0.234375
step 6000, training accuracy 0.28125
step 7000, training accuracy 0.25
step 8000, training accuracy 0.203125
step 9000, training accuracy 0.265625


In [5]:
for i in range(10):
    x_test_batch, y_test_batch = mnist.test.next_batch(batch_size)
    test_accuracy = sess.run(accuracy,feed_dict={x:x_test_batch,y_:y_test_batch,keep_prob:1.0})
    print('step %d, testing accuracy %g' % (i,test_accuracy))  
    

step 0, testing accuracy 0.265625
step 1, testing accuracy 0.25
step 2, testing accuracy 0.25
step 3, testing accuracy 0.25
step 4, testing accuracy 0.171875
step 5, testing accuracy 0.296875
step 6, testing accuracy 0.25
step 7, testing accuracy 0.28125
step 8, testing accuracy 0.25
step 9, testing accuracy 0.265625
